# Explaining how to solve a Sudoku

This notebook covers how to generate a sequence of explanations for Constraint Satisfaction problems. Here the use case is: 

You are solving a Sudoku, but at one point, you don't know how to continue. Is there a _HINT_ button I can press to help me out and tell me which number I should write next? 

The answer is YES! 

In this notebook, we present how the reasoning behind this button works in practice. We show how to generate these ___explanations___ and how you can adapt them in order to fit your preferences.

First we load the CPMpy library:


In [1]:
from cpmpy import *
from cpmpy.transformations.get_variables import get_variables

import numpy as np
from time import time
from tqdm import tqdm

from IPython.display import display, HTML # display some titles

## 1. Modeling the Sudoku

In practice, some puzzlers like annotate their sudoku with the numbers that cannot be selected. To keep the explanations limited,in this notebook, we model the sudoku using integer variables and consider only explanations consisting of positive assignments. 

In [2]:
def model_sudoku(given):
    # Dimensions of the sudoku problem
    ncol = nrow = len(given)
    n = int(ncol ** (1/2))

    # Model Variables
    cells = intvar(1,ncol,shape=given.shape,name="cells")
    
    # sudoku must start from initial clues
    facts = [cells[given != e] == given[given != e]]
    # rows constraint
    row_cons = [AllDifferent(row) for row in cells]
    # columns constraint
    col_cons = [AllDifferent(col) for col in cells.T]
    # blocks constraint
    # Constraints on blocks
    block_cons = []

    for i in range(0,ncol, n):
        for j in range(0,nrow, n):
            block_cons += [AllDifferent(cells[i:i+n, j:j+n])]
            
    assert Model(facts + row_cons + col_cons + block_cons).solve()

    return cells, facts, row_cons + col_cons + block_cons

Pretty printing of a sudoku grid

In [3]:
def print_sudoku(grid):
    out = ""
    nrow = ncol = len(grid)
    n = int(nrow ** (1/2))
    for r in range(0,nrow):
        for c in range(0,ncol):
            out += str(grid[r, c] if grid[r, c] > 0 else ' ')
            out += '  ' if grid[r, c] else '  '
            if (c+1) % n == 0 and c != nrow-1: # end of block
                out += '| '
        out += '\n'
        if (r+1) % n == 0 and r != nrow-1: # end of block
            out += ('-'*(n + 2*n))
            out += ('+' + '-'*(n + 2*n + 1)) *(n-1) + '\n'
    print(out)   

# Example instances

In [4]:
e = 0
given_4x4 = np.array([
    [ e, 3, 4, e ],
    [ 4, e, e, 2 ],
    [ 1, e, e, 3 ],
    [ e, 2, 1, e ],
])
print_sudoku(given_4x4)

   3  | 4     
4     |    2  
------+-------
1     |    3  
   2  | 1     



## Example SUDOKU: Model and Solve

The following grid is an example of a Sudoku grid, where the given values have a value greater than 0 and the others are fixed to 0.

In [5]:
e = 0
given_9x9 = np.array([
    [e, e, e,  2, e, 5,  e, e, e],
    [e, 9, e,  e, e, e,  7, 3, e],
    [e, e, 2,  e, e, 9,  e, 6, e],

    [2, e, e,  e, e, e,  4, e, 9],
    [e, e, e,  e, 7, e,  e, e, e],
    [6, e, 9,  e, e, e,  e, e, 1],

    [e, 8, e,  4, e, e,  1, e, e],
    [e, 6, 3,  e, e, e,  e, 8, e],
    [e, e, e,  6, e, 8,  e, e, e]])

print_sudoku(given_9x9)

         | 2     5  |          
   9     |          | 7  3     
      2  |       9  |    6     
---------+----------+----------
2        |          | 4     9  
         |    7     |          
6     9  |          |       1  
---------+----------+----------
   8     | 4        | 1        
   6  3  |          |    8     
         | 6     8  |          



## Solving the Sudoku

Call the solver on the Sudoku model and extract the solution.

In [6]:
def extract_solution(constraints, variables, verbose=False):
    ncols = nrows = 9
    solved = Model(constraints).solve()
    assert solved, "Model is unsatisfiable."
    
    sol = np.array([
        [variables[r, c].value() 
         for c in range(ncols)] 
        for r in range(nrows)
    ])
     
    return sol

In [7]:
e = 0 # value for empty cells
given = np.array([
    [6, 9, 4,  e, e, 1,  e, e, e],
    [e, e, 3,  e, 2, e,  e, 4, 5],
    [2, 7, e,  e, 6, e,  e, e, e],

    [e, e, 1,  e, e, 4,  e, 7, e],
    [e, 2, 6,  e, 8, 7,  e, 9, 3],
    [3, 5, 7,  e, e, e,  4, e, 2],

    [e, 6, 9,  e, e, e,  e, e, 1],
    [1, 3, e,  6, e, e,  7, e, e],
    [e, e, e,  1, e, 2,  e, e, e]]
)

display(HTML('<h3> ------- INPUT SUDOKU -------</h3>'))
print_sudoku(given)

sudoku_vars, facts, constraints = model_sudoku(given)

sudoku_solution = extract_solution(constraints + facts, sudoku_vars)

display(HTML('<h3>---------- SOLUTION ---------- </h3>'))
print_sudoku(sudoku_solution)

6  9  4  |       1  |          
      3  |    2     |    4  5  
2  7     |    6     |          
---------+----------+----------
      1  |       4  |    7     
   2  6  |    8  7  |    9  3  
3  5  7  |          | 4     2  
---------+----------+----------
   6  9  |          |       1  
1  3     | 6        | 7        
         | 1     2  |          



6  9  4  | 8  5  1  | 2  3  7  
8  1  3  | 7  2  9  | 6  4  5  
2  7  5  | 4  6  3  | 9  1  8  
---------+----------+----------
9  8  1  | 2  3  4  | 5  7  6  
4  2  6  | 5  8  7  | 1  9  3  
3  5  7  | 9  1  6  | 4  8  2  
---------+----------+----------
7  6  9  | 3  4  5  | 8  2  1  
1  3  2  | 6  9  8  | 7  5  4  
5  4  8  | 1  7  2  | 3  6  9  



## Explanations for SUDOKU: 

To explain a full sudoku from the givens to the solution, we generate a sequence of intermediate expalnation steps.
Every explanation step is characterized by an *interpretation*, which corresponds to current status of the grid. 
The initial state of the gris is called the ***initial interpretation***, and the solution is also known as the ***end interpretation***.

Every explanation step uses subset of the problem constraints and part of the interpretation in order to derive one or multiple new numbers. 

1. __C__' ⊂ __C__ A subset of the problem constraints (alldifferents on columns, rows and blocks).

2. __I__' ⊂ __I__ A partial interpretation

    - In the Sudoku case this corresponds to the numbers filled in the grid at the current explanation step (givens, and newly derived numbers).

3. __N__ A newly found number to fill in the grid.

Therefore at every step, the number 

To compute such explanations, we take advantage of the link between non-redundant explanations and Minimal Unsatisfiable Subsets introduced in [1]. 


[1] Bogaerts, B., Gamba, E., & Guns, T. (2021). A framework for step-wise explaining how to solve constraint satisfaction problems. Artificial Intelligence, 300, 103550.


In [8]:
class SMUSAlgo:

    def __init__(self, soft, hard=[], solvername="ortools", hs_solvername="gurobi", disable_corr_enum=False):

        self.soft = cpm_array(soft)
        self.hard = hard
        
        self.assum_vars = boolvar(len(soft))

        self.solver = SolverLookup.get(solvername)
        self.solver += self.hard
        self.solver += self.assum_vars.implies(self.soft)
        
        self.maxsat_solver = SolverLookup.get(solvername)
        self.maxsat_solver += self.hard
        self.maxsat_solver += self.assum_vars.implies(self.soft)
        self.maxsat_solver.maximize(sum(self.assum_vars))

        self.hs_solver= SolverLookup.get(hs_solvername)
        self.hs_solver.minimize(sum(self.assum_vars))

        self.mus = None
        self.disable_corr_enum = disable_corr_enum
        
    def iterate(self, n=1):
        for _ in range(n):
            assert self.hs_solver.solve()
            h = self.assum_vars.value() == 1
            
            if not self.solver.solve(assumptions=self.assum_vars[h]):
                # UNSAT subset, return
                self.mus = set(self.soft[self.assum_vars.value()])
                return
                
            # Find disjunctive sets
            mcses = self.corr_enum(h)

            for mcs in mcses:
                self.hs_solver += sum(self.assum_vars[mcs]) >= 1

    def corr_enum(self, h):
           
        mcses = []
        hp = np.array(h)

        sat, ss = self.grow(h)
        
        if self.disable_corr_enum:
            return [~ss]

        while(sat):
            mcs = ~ss
            mcses.append(mcs)
            hp = hp | mcs
            sat, ss = self.grow(hp)

        return mcses            
    
    def grow(self, h):
#         from time import time
        start = time()
        sat = self.maxsat_solver.solve(assumptions=self.assum_vars[h])

        return sat, self.assum_vars.value() == 1

## Explanation
This part of the notebook finds the smallest step to take in solving the sudoku. <br>
For this, we build upon out work on stepwise explanations of SAT problems [2].

In [9]:
def split_ocus(given, vars_to_expl, constraints, verbose=False,disable_corr_enum=False):
    facts = [var == val for var, val in given.items()]
    assert Model(facts + constraints).solve(), "Model should be SAT!"
    sol = {var : var.value() for var in vars_to_expl}
    
    pq = [(var,0,SMUSAlgo(soft=facts + constraints, hard=[var != sol[var]], disable_corr_enum=disable_corr_enum)) 
          for var in vars_to_expl]
    
    i = 0
    while 1:
        var, obj_val, smus_algo = pq.pop(0)
        if verbose:
            print(f"\rContinuing computation on SMUS with obj {obj_val}", end="\t"*5)
        # pbar.set_description(f"Best objective_score: {obj_val}")
        if smus_algo.mus is not None:
            E = smus_algo.mus & set(facts)
            S = smus_algo.mus & set(constraints)
            N = [var == sol[var]]
            return (E,S,get_variables(N))
        # Algo has not found a solution yet, continue
        smus_algo.iterate()
        pq.append((var,smus_algo.hs_solver.objective_value(),smus_algo))
        pq.sort(key=lambda x : x[1])
        # pbar.update()

### Propagation of constraints 

Once an explanation is found, we check whether we can derive some other digits using the same constraints and the digits of the explanation.

In [10]:
def explain_ocus(given, vars_to_expl, constraints, verbose=False):
    facts = [var == val for var, val in given.items()]
    assert Model(facts + constraints).solve(), "Model should be SAT!"
    sol = {var : var.value() for var in vars_to_expl}
    
    soft = facts + constraints + [var != var.value() for var in vars_to_expl]
    cons_subset = np.append(np.zeros(len(facts) + len(constraints), dtype=bool),
                            np.ones(len(vars_to_expl), dtype=bool))
    
    ocus_algp = OCUS_OneOfAlgo(soft, cons_subset)
    
    mus = ocus_algp.iterate(int(1e3))

    E = ocus_algp.mus & set(facts)
    S = ocus_algp.mus & set(constraints)
    N = ocus_algp.mus - E - S
    return (E,S,get_variables(N))

In [11]:
def split_ocus_subsetmax(given, vars_to_expl, constraints, verbose=False):
    facts = [var == val for var, val in given.items()]
    assert Model(facts + constraints).solve(), "Model should be SAT!"
    sol = {var : var.value() for var in vars_to_expl}
    
    pq = [(var,0,SMUSAlgo_subsetmax(soft=facts + constraints, hard=[var != sol[var]])) for var in vars_to_expl]
    i = 0
    while 1:
        var, obj_val, smus_algo = pq.pop(0)
        if verbose:
            print(f"\rContinuing computation on SMUS with obj {obj_val}", end="\t"*5)
        # pbar.set_description(f"Best objective_score: {obj_val}")
        if smus_algo.mus is not None:
            E = smus_algo.mus & set(facts)
            S = smus_algo.mus & set(constraints)
            N = [var == sol[var]]
            return (E,S,get_variables(N))
        # Algo has not found a solution yet, continue
        smus_algo.iterate()
        pq.append((var,smus_algo.hs_solver.objective_value(),smus_algo))
        pq.sort(key=lambda x : x[1])
        # pbar.update()

In [12]:
def split_ocus_subsetmax_corr_enum(given, vars_to_expl, constraints, verbose=False):
    facts = [var == val for var, val in given.items()]
    assert Model(facts + constraints).solve(), "Model should be SAT!"
    sol = {var : var.value() for var in vars_to_expl}
    
    pq = [(var,0,SMUSAlgo_subsetmax_corr_enum(soft=facts + constraints, hard=[var != sol[var]])) for var in vars_to_expl]
    i = 0
    while 1:
        var, obj_val, smus_algo = pq.pop(0)

        if verbose:
            print(f"\rContinuing computation on SMUS with obj {obj_val}", end="\t"*5)
            
        # pbar.set_description(f"Best objective_score: {obj_val}")
        if smus_algo.mus is not None:
            E = smus_algo.mus & set(facts)
            S = smus_algo.mus & set(constraints)
            N = [var == sol[var]]
            return (E,S,get_variables(N))

        # Algo has not found a solution yet, continue
        smus_algo.iterate()
        pq.append((var,smus_algo.hs_solver.objective_value(),smus_algo))
        pq.sort(key=lambda x : x[1])
        # pbar.update()

In [13]:
class SMUSAlgo_subsetmax(SMUSAlgo):
    def __init__(self, soft, hard=[], solvername="ortools", hs_solvername="gurobi"):
        self.soft = cpm_array(soft)
        self.hard = hard
        
        self.assum_vars = boolvar(len(soft))

        self.solver = SolverLookup.get(solvername)
        self.solver += self.hard
        self.solver += self.assum_vars.implies(self.soft)

        self.hs_solver= SolverLookup.get(hs_solvername)
        self.hs_solver.minimize(sum(self.assum_vars))

        self.mus = None
    
    
    def grow(self, h):

        
        sat = self.solver.solve(assumptions=self.assum_vars[h])
        
        ss = self.assum_vars.value() == 1

        remaining_to_check = set(i for i, v in enumerate(ss) if not v)
        
        while(len(remaining_to_check) > 0):
            i = remaining_to_check.pop()
            
            ssp = np.concatenate([ss[:i], [True], ss[i+1:]])

            if self.solver.solve(assumptions=self.assum_vars[ssp]):
                ss |= (self.assum_vars.value() == 1)
                remaining_to_check -= set(i for i, v in enumerate(ss) if v)

        return ss


In [14]:
class SMUSAlgo_subsetmax_corr_enum(SMUSAlgo):
    def __init__(self, soft, hard=[], solvername="ortools", hs_solvername="gurobi"):
        super().__init__(soft, hard, solvername, hs_solvername)
            
    def grow(self, h):

        sat = self.solver.solve(assumptions=self.assum_vars[h])

        if not sat:
            return sat, h

        ss = self.assum_vars.value() == 1
        
        for i, val in enumerate(np.array(ss)):
            if val:
                continue
            ssp = np.concatenate([ss[:i], [not val], ss[i+1:]])

            if self.solver.solve(assumptions=self.assum_vars[ssp]):
                ss = ssp

        # self.maxsat_solver.solve(assumptions=self.assum_vars[h])
        return sat, ss == 1

In [15]:
class OCUS_OneOfAlgo(SMUSAlgo):

    def __init__(self, soft, cons_subset, hard=[], solvername="ortools", hs_solvername="gurobi"):
        super().__init__(soft, hard, solvername, hs_solvername)

        self.hs_solver += sum(self.assum_vars[cons_subset]) == 1
        # self.maxsat_solver += sum(self.assum_vars[cons_subset]) == 1

## Run Stepwise explanations

In [16]:
given = given_4x4
#given = given_9x9
cells, facts, constraints = model_sudoku(given)

clues = {var : val for var, val in zip(cells.flatten(), given.flatten()) if val != e}
vars_to_expl = set(get_variables(constraints)) - set(clues.keys())

start = time()
E,S,N = split_ocus(clues, vars_to_expl, constraints, verbose=True, disable_corr_enum=True)

print("\n Split OCUS - MaxSAT Corr Enum", "\t"*4, f"{time()-start}s")

Restricted license - for non-production use only - expires 2023-10-25
Continuing computation on SMUS with obj 7.0					
 Split OCUS - MaxSAT Corr Enum 				 281.7976357936859s


In [17]:
# helper function
from cpmpy.expressions.utils import is_any_list
def _unravel(lst):
    if is_any_list(lst):
        flatcons = [_unravel(e) for e in lst]
        return [c for con in flatcons for c in con]
    return [lst]

# OCUS-based explanations

In [18]:
# start = time()

# E,S,N = split_ocus_subsetmax(clues, vars_to_expl, constraints)

# print("\n Split OCUS - SubsetMAX", "\t"*4, f"{time()-start}s")

# Adding correction subset enum for OCUS-based explanations

In [19]:
# start = time()

# E,S,N = split_ocus_subsetmax_corr_enum(clues, vars_to_expl, constraints)

# print("\n Split OCUS - Subsetmax Corr Enumeration", "\t"*4, f"{time()-start}s")

In [20]:
# start = time()
# E,S,N = split_ocus_subsetmax(clues, vars_to_expl, constraints)
# print("\n Split OCUS - Subsetmax", "\t"*4, f"{time()-start}s")